# Using FalkorGraphRagCapability with agents for GraphRAG Question & Answering

AG2 provides GraphRAG integration using agent capabilities. This is an example to integrate FalkorDB (a Knowledge Graph database).

## Install FalkorDB SDK

In [6]:
%pip install graphrag_sdk==0.3.3
%pip install -e /workspaces/ag2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///workspaces/ag2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ag2 (pyproject.toml) ... done
  Created wheel for ag2: filename=ag2-0.4-0.editable-py3-none-any.whl size=15779 sha256=6352d50605c2b333aabeb86bee921ade8e34d3429ed67f175db6d0d23dcacd05
  Stored in directory: /tmp/pip-ephem-wheel-cache-62aq_s_d/wheels/81/96/a6/c21a93c37cecb7b27b25b264f04fa27be94acd8d0f5e637527
Successfully built ag2
  Attempting uninstall: ag2
    Found existing installation: ag2 0.4
    Uninstalling ag2-0.4:
      Successfully uninstalled ag2-0.4
Note: you may need to restart the kernel to use

## Set Configuration and OpenAI API Key

In order to use FalkorDB you need to have an OpenAI key in your environment variable `OPENAI_API_KEY`.

You can utilise an OAI_CONFIG_LIST file and extract the OpenAI API key and put it in the environment, as will be shown in the following cell.

Alternatively, you can load the environment variable yourself.

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

In [3]:
import os

import autogen

config_list = autogen.config_list_from_json(env_or_file="../OAI_CONFIG_LIST")

# Put the OpenAI API key into the environment
os.environ["OPENAI_API_KEY"] = config_list[0]["api_key"]

````{=mdx}
:::important
The default model for loading graph data and answering questions using FalkorDB's SDK is OpenAI's GPT 4o and this can be changed by setting the `model` parameter on the FalkorGraphQueryEngine.
:::
````

## Create a Knowledge Graph with Your Own Data

**Note:** You need to have a FalkorDB graph database running. If you are running one in a Docker container, please ensure your Docker network is setup to allow access to it.

In this example, the FalkorDB endpoint is set to host="172.18.0.3" and port=6379, please adjust accordingly. For how to set up FalkorDB, please refer to https://docs.falkordb.com/


Below, we have some sample data from IMDB on the movie 'The Matrix'.

We then initialise the database with that text document, creating the graph in FalkorDB.

### A Simple Example
In this example, the graph ontology is auto-generated.

In [ ]:
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.graph_rag.document import Document, DocumentType
from autogen.agentchat.contrib.graph_rag.falkor_graph_query_engine import FalkorGraphQueryEngine
from autogen.agentchat.contrib.graph_rag.falkor_graph_rag_capability import FalkorGraphRagCapability

# Auto generate graph schema from unstructured data
input_path = "../test/agentchat/contrib/graph_rag/the_matrix.txt"
input_documents = [Document(doctype=DocumentType.TEXT, path_or_url=input_path)]

# Create FalkorGraphQueryEngine
query_engine = FalkorGraphQueryEngine(
    name="The_Matrix_Auto",
    host="172.18.0.3",  # Change
    port=6379,  # if needed
)

# Ingest data and initialize the database
query_engine.init_db(input_doc=input_documents)

# Create a ConversableAgent (no LLM configuration)
graph_rag_agent = ConversableAgent(
    name="matrix_agent",
    human_input_mode="NEVER",
)

# Associate the capability with the agent
graph_rag_capability = FalkorGraphRagCapability(query_engine)
graph_rag_capability.add_to_agent(graph_rag_agent)

# Create a user proxy agent to converse with our RAG agent
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
)

user_proxy.initiate_chat(graph_rag_agent, message="Name a few actors who've played in 'The Matrix'")

user_proxy (to matrix_agent):

Name a few actors who've played in 'The Matrix'

--------------------------------------------------------------------------------
matrix_agent (to user_proxy):

Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, and Hugo Weaving all played in 'The Matrix'.

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': "Name a few actors who've played in 'The Matrix'", 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, and Hugo Weaving all played in 'The Matrix'.", 'role': 'user', 'name': 'matrix_agent'}], summary="Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, and Hugo Weaving all played in 'The Matrix'.", cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=['exit'])

### Revisit the exmple with self defined graph ontology.
We define the entities and attributes that we want in the graph database (`Actor` with `name` and `Movie` with `title`) and the relationships (Actors `ACTED` in Movies). This allows your RAG agent to answer questions about actors in the movie.

In [5]:
from graphrag_sdk import Attribute, AttributeType, Entity, Ontology, Relation

from autogen.agentchat.contrib.graph_rag.document import Document, DocumentType
from autogen.agentchat.contrib.graph_rag.falkor_graph_query_engine import FalkorGraphQueryEngine

input_path = "../test/agentchat/contrib/graph_rag/the_matrix.txt"

movie_ontology = Ontology()
movie_ontology.add_entity(
    Entity(label="Actor", attributes=[Attribute(name="name", attr_type=AttributeType.STRING, unique=True)])
)
movie_ontology.add_entity(
    Entity(label="Movie", attributes=[Attribute(name="title", attr_type=AttributeType.STRING, unique=True)])
)
movie_ontology.add_relation(Relation(label="ACTED", source="Actor", target="Movie"))

query_engine = FalkorGraphQueryEngine(
    name="IMDB",
    host="172.18.0.3",  # Change
    port=6379,  # if needed
    ontology=movie_ontology,
)

input_documents = [Document(doctype=DocumentType.TEXT, path_or_url=input_path)]

query_engine.init_db(input_doc=input_documents)

## Add capability to a ConversableAgent and query them

With FalkorDB setup, we can now associate the GraphRAG capability with a ConversableAgent and have a chat with it.

In [3]:
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.graph_rag.falkor_graph_rag_capability import FalkorGraphRagCapability

# Create a ConversableAgent (no LLM configuration)
graph_rag_agent = ConversableAgent(
    name="matrix_agent",
    human_input_mode="NEVER",
)

# Associate the capability with the agent
graph_rag_capability = FalkorGraphRagCapability(query_engine)
graph_rag_capability.add_to_agent(graph_rag_agent)

# Create a user proxy agent to converse with our RAG agent
user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config=False,
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    human_input_mode="ALWAYS",
)

user_proxy.initiate_chat(graph_rag_agent, message="Name a few actors who've played in 'The Matrix'")

# You will be prompted, as a human in the loop, after the response - feel free to ask more questions.

user_proxy (to matrix_agent):



Name a few actors who've played in 'The Matrix'

--------------------------------------------------------------------------------
matrix_agent (to user_proxy):

Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, and Hugo Weaving have played in 'The Matrix'.

--------------------------------------------------------------------------------
user_proxy (to matrix_agent):

List additional actors.

--------------------------------------------------------------------------------
matrix_agent (to user_proxy):

The actors Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, and Hugo Weaving are listed. There are no additional actors provided in the context related to 'The Matrix'.

--------------------------------------------------------------------------------
user_proxy (to matrix_agent):

List additional people associated with the movie.

--------------------------------------------------------------------------------
matrix_agent (to user_proxy):

I am sorry, I could not find the answer to 

ChatResult(chat_id=None, chat_history=[{'content': "Name a few actors who've played in 'The Matrix'", 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, and Hugo Weaving have played in 'The Matrix'.", 'role': 'user', 'name': 'matrix_agent'}, {'content': 'List additional actors.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "The actors Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, and Hugo Weaving are listed. There are no additional actors provided in the context related to 'The Matrix'.", 'role': 'user', 'name': 'matrix_agent'}, {'content': 'List additional people associated with the movie.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'I am sorry, I could not find the answer to your question', 'role': 'user', 'name': 'matrix_agent'}], summary='I am sorry, I could not find the answer to your question', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {